convert to presentation: jupyter nbconvert LSMC_explanation.ipynb --to slides --no-prompt
jupyter nbconvert LSMC_explanation.ipynb --to slides --post serve
https://digitalhumanities.hkust.edu.hk/tutorials/turn-your-jupyter-notebook-into-interactive-presentation-slides-using-anaconda/
https://digitalhumanities.hkust.edu.hk/tutorials/how-to-host-your-website-on-github-page/

In [1]:
from IPython.display import display, HTML

CSS = """
.output {
    align-items: center;
}
"""

HTML('<style>{}</style>'.format(CSS))

# **Implementation of Least Squares Monte Carlo**
****
### Using Longstaff-Schwartz algorithm

***This slide will be skipped***

Our implementation of the Least Squares Monte Carlo method for pricing american options is based on the work of Longstaff and Schwartz, presented in Longstaff and Schwartz (2001). 

American options call for a significant deviation from European or Asian-style pricing techniques, they allow for an early exercise before their maturity. This feature implies that when computing their value, we have to take into account the fact that a rational investor will continuously choose wether to exercise or hold the option, based on which choice will maximize her expected value. An American-style option pricing model must therefore compute the value of the option at every time step as the maximum between the payoff of exercising the option at that time step (early exercise value) and the expected value of holding the option to maturity (continuation value).



## Longstaff-Schwartz algorithm
- Proposed in **Longstaff and Schwartz (2001)**.
- Computes price of options that can be exercised before maturity.
- Uses least-squares regression to estimate conditional expectations.
- Useful to price American-style options.

## American Options
Allow for **early exercise**: 

The value at each time step is the maximum between the payoff of exercising the option at that time step (early exercise value) and the expected value of holding the option to maturity (continuation value).

For an **American call** option:
$$
V_t^i = \max\left((S_t^i - K)^+,\: e^{-(T-t)r}(E[S_T|S_t^i] - K)^+ \right)
$$

## Motivation: Why do we need the LSMC algorithm?

- How can we evaluate the **continuation value** $E[S_T|S_t^i]$?

    Nested Monte Carlo simulations for every $S_t^i$ until maturity &rarr; **unfeasible** for large numbers
    
- Binomial Tree: high discretization error if used with long time steps.

    Will **underestimate** the number of **early exercise opportunities** as it only provides two outcomes for the value of the underlying.

    On the other hand, time complexity: $O(2^n)$
    

The first step in the Longstaff-Schwartz algorithm is to determine the expected payoff at maturity. This can be computed as the payoff of a conventional European option, as the continuation value is zero.
$$
E[\pi_T] = \max(E[S_{T}] - K,0)
$$
Then, we move to time step $T-1$, where the holder must decide wether to exercise the option or hold it until expiration date T. At time T-1, 


## LSMC algorithm
We take as example an **American call** option with 1 year maturity, exercisable at times 1,2 and 3. Furthermore: 

$$
S_0 = 1,\: K = 1.1,\: r = 0.1,\: \sigma = 0.2
$$

In [1]:
import numpy as np
import pandas as pd
import Price_Models as pm
import scipy.stats as stats

# Compute price of an american put option with the following parameters
S_0, K, T, sigma, r = 1,1.1,1,0.2,0.1
M = 8

# generate the stock price paths
N = 3 #Time steps with trading days
dt = T / N
S = pm.BS_path(S_0, r, sigma, T, N, nPaths = M)
S_paths = pd.DataFrame(S)
S_paths = S_paths.rename(columns={0: 't = 0',  1: 't = 1',  2: 't = 2', 3: 't = 3'})
S_paths = S_paths.round(decimals = 2)

In [6]:
import dataframe_image as dfi
dfi.export(S_paths, './Presentation files/Stock_path.png', dpi = 300)

In [2]:
S_paths

,t = 0,t = 1,t = 2,t = 3
0,1.0,1.15,0.92,1.05
1,1.0,1.20,1.19,1.33
2,1.0,0.96,0.97,1.04
3,1.0,0.96,1.19,1.46
4,1.0,1.04,1.02,0.99
5,1.0,1.07,1.02,1.08
6,1.0,1.13,1.18,1.36
7,1.0,1.08,1.18,1.19


## LSMC algorithm: Step 1
- Determine the expected payoff at maturity: since continuation value is zero, it is the payoff of a vanilla European option

In [7]:
payoff = np.maximum(S - K, 0)
dis_cfl = np.zeros((M, N+1)) # prepare matrix for discounted cashflow at every timestep 
dis_cfl[:,N] = payoff[:,N]
dfi.export(pd.DataFrame(dis_cfl).round(decimals = 2), './Presentation files/Payoff_maturity.png', dpi = 300)

pd.DataFrame(dis_cfl).round(decimals = 2)

,0,1,2,3
0,0.0,0.0,0.0,0.00
1,0.0,0.0,0.0,0.23
2,0.0,0.0,0.0,0.00
3,0.0,0.0,0.0,0.36
4,0.0,0.0,0.0,0.00
5,0.0,0.0,0.0,0.00
6,0.0,0.0,0.0,0.26
7,0.0,0.0,0.0,0.09


## LSMC algorithm: Step 2
- One time step back: consider the paths were the option is **in the money** \
        &rarr; The holder must decide to exercise or not
- Discount the payoff of holding the option to maturity: $y_{t=2}^i = e^{-r}\pi_{t=3}^i$
- Get value of underlying at time T-1

In [16]:
exercise_flag = np.zeros((M,N)) # should we exercise
cond = S[:,-2] > K # in the money
Y = np.exp(-r*dt) * dis_cfl[cond,-1]
S_2 = S[cond,-2]
print(Y, S_2)
display_df = pd.DataFrame({"Disc val":Y}).assign(S_2=S_2)
display_df
display(display_df.round(decimals = 2))
dfi.export(display_df.round(decimals = 2), './Presentation files/Disc_payoff.png', dpi = 300)
exercise_flag[cond, -1] = 1
# pd.DataFrame(exercise_flag)


[0.22689538 0.3479476  0.25144297 0.08994861] [1.18564494 1.18736242 1.17527395 1.17641294]


,Disc val,S_2
0,0.23,1.19
1,0.35,1.19
2,0.25,1.18
3,0.09,1.18


[0.91828535 1.18564494 0.97207179 1.18736242 1.01925776 1.0228439
 1.17527395 1.17641294] [False  True False  True False False  True  True]


## LSMC algorithm: Step 3
- Regress $y_{t=2}$ on a set of basis functions of $S_{t=2}$

If $a_j$ are coefficients and $B_j$ is the set of basis functions, then the **continuation value** for a path $i$ with values $S_{i,t_n}$ at time $t_{n}$ is

\begin{equation}
\begin{array}{lll}
Cont_t^i& =& \sum_{j=0}^{\infty} a_j\left(t_{n}\right)B_j(S_{t})\\
\end{array}
\end{equation}


In [29]:
X = np.column_stack([np.ones(8), S[:,-2], S[:,-2]**2, S[:,-2]**3,S[:,-2]**4,S[:,-2]**5]) # Create matrix of basis functions for regressions
cond_x = X[cond, :]
beta = np.linalg.lstsq(cond_x, Y, rcond=None)[0]
continue_val = np.dot(X, beta)
continue_val[~cond] = 0
continue_val
Continue_df = pd.DataFrame({"Continue val":continue_val[cond], "P_2":payoff[:,-2][cond]})

dfi.export(Continue_df.round(decimals = 3), './Presentation files/Continue_payoff.png', dpi = 300)
Continue_df

,Continue val,P_2
0,0.226895,0.085645
1,0.347948,0.087362
2,0.251443,0.075274
3,0.089949,0.076413


In [ ]:
payoff = np.maximum(S - K, 0)

# perform the least squares regression
dis_cfl = np.zeros((M, N+1)) # discounted cashflow at every timestep 
dis_cfl[:,N] = payoff[:,N] 
exercise_flag = np.zeros((M,N)) # should we exercise
cond = S[:,-1] > K # not in the money
exercise_flag[cond, -1] = 1
for i in range(N-1, 0, -1): # backward
    cond = S[:,i] > K
    X = np.column_stack([np.ones(M), S[:,i], S[:,i]**2])
    cond_x = X[cond, :]
    Y = np.exp(-r*dt) * dis_cfl[cond,i+1]
    beta = np.linalg.lstsq(cond_x, Y, rcond=None)[0]
    continue_val = np.dot(X, beta)
    continue_val[~cond] = 0
    cond_exercise = payoff[:,i] > continue_val
    exercise_flag[cond_exercise, i-1] = 1
    dis_cfl[:,i] = np.exp(-r*dt) * dis_cfl[:,i+1]
    dis_cfl[cond_exercise,i] = payoff[cond_exercise,i]

stopping_criteria = np.argmax(exercise_flag, axis=1) # first exercise point

actual_exercise = np.zeros_like(exercise_flag)
actual_exercise[np.arange(M), stopping_criteria] = exercise_flag[np.arange(M), stopping_criteria]
discount = (np.ones((M, N))*np.exp(-r*dt)).cumprod(axis=1)[::-1]
exp_payoff = (actual_exercise * payoff[:,1:] * discount).sum() / M
exp_payoff